<a href="https://colab.research.google.com/github/jithinprakashk/Miscellaneous/blob/master/Covide19_Cases_India_Using_Folium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Import Libraries
import folium
import pandas as pd
import matplotlib.pyplot as plt
import requests 
from  geopy.geocoders import Nominatim
 
 
# Covid 19 website ## Api to fetch the covid 19 data
url = 'https://api.covid19india.org/data.json'  
 
# make a GET request to fetch the raw content in json format and pushing to DF
web_content = requests.get(url).json() 
statedata = web_content['statewise']
state_df = pd.DataFrame(statedata)[['state','active','confirmed','deaths','recovered','statecode']].iloc[1:,:] #
 
#Finding Latitude and longitude
geolocator = Nominatim()
state_df['lat'] = state_df['state'].apply(lambda x : geolocator.geocode(x +',India'))
state_df.dropna(axis=0,inplace=True)
state_df['long'] =state_df['lat'].apply(lambda x : tuple(x)[-1][1])
state_df['lat'] =state_df['lat'].apply(lambda x : tuple(x)[-1][0])

/usr/local/lib/python3.6/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


In [0]:
india_map = folium.Map(location=[20.5937, 78.9629], zoom_start=4.5)
 
for state in state_df["state"]:
    datapoint = state_df[state_df["state"]==state]
    pp=state + '\n\nActive Case:'+ str(datapoint.active.values[0]) + '\nConfirmed Case:' + str(datapoint.confirmed.values[0]) + \
                      '\nDeaths :'+ str(datapoint.deaths.values[0]) +'\nRecovered :' + str(datapoint.recovered.values[0])
    divisor = 1500 if int(datapoint["active"].values[0])>10000 else 500
    div = 1 if int(datapoint["active"].values[0])==0 else int(datapoint["active"].values[0])
    folium.CircleMarker(
        location=[datapoint.lat.values[0], datapoint.long.values[0]],
        radius = float(div/divisor),
        popup = pp ,
        tooltip = state+ '\n'+str(datapoint.active.values[0]),
        color="Red",
        fill_color="red"
    ).add_to(india_map)
 
india_map